In [30]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-70b-8192",   
    temperature=0, 
    groq_api_key="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"
)

In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.livehire.com/careers/nikeinc/job/6HN4T/5VETK8DP16/software-devops-engineer")
page_data=loader.load().pop().page_content  
print(page_data)  





Software/DevOps Engineer - Nike | Humanforce Talent



















































Log inHumanforceLinkedInFacebookGoogleCancelPowered by‹ Back to all jobs Software/DevOps EngineerHybridLocation1055 Howell Mill Rd, Atlanta, GA 30318, USAWork TypeContract/TempPositions1 PositionPublished At:11 hours agoSoftware EngineerAWSCloud pythonDevOpsAzureTerraformCloud PlatformsRefer someoneApplyOnsite: Mon-ThursCategory: TechnologyInnovative Technology; High Quality Products, Self-EmpowermentGlobally Responsible; Sustainable Products, Diversity of ThoughtCelebration of Sports; If You Have a Body, You are an AthleteTitle: Software/DevOps EngineerLocation: Atlanta, GADuration: 12 month contractNIKE, Inc. does more than outfit the world's best athletes. It is a place to explore potential, obliterate boundaries and push out the edges of what can be. The company looks for people who can grow, think, dream and create. Its culture thrives by embracing diversity and rewarding ima

In [31]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """ 
    ## SCRAPED TEXT FROM WEBSITE: 
    {page_data}
    The scraped text is from career's page of a website.
    Your job is to extract the job postings and return them in json format containing the following
    keys:role,skills,description.
    Return them in valid json. 
    ### NO PREAMBLE :
    """ 
)
chain_extract=prompt_extract | llm 
res=chain_extract.invoke(input={'page_data':page_data})
print(res.content) 

Here is the extracted job posting in JSON format:

```
[
    {
        "role": "Software/DevOps Engineer",
        "skills": [
            "AWS",
            "Cloud",
            "python",
            "DevOps",
            "Azure",
            "Terraform",
            "Cloud Platforms",
            "Python/Golang",
            "Infrastructure-as-code",
            "Cloud security",
            "Agile",
            "JIRA",
            "Cloud governance",
            "Cloud Custodian",
            "Open Policy Agent Rego",
            "Cloud Security/Cybersecurity",
            "Scaled Agile Framework (SAFe)",
            "AWS Certifications"
        ],
        "description": "We are looking for a Software/DevOps engineer with the skills to automate repeatable processes to improve the efficiency of our cloud governance capabilities, help develop integrated solutions, and assist in the delivery of policy-as-code across a multi-cloud ecosystem."
    }
]
```

Let me know if you need any fur

In [32]:
from langchain_core.output_parsers import JsonOutputParser 
json_parser=JsonOutputParser ()
json_res=json_parser.parse(res.content) 
json_res 

[{'role': 'Software/DevOps Engineer',
  'skills': ['AWS',
   'Cloud',
   'python',
   'DevOps',
   'Azure',
   'Terraform',
   'Cloud Platforms',
   'Python/Golang',
   'Infrastructure-as-code',
   'Cloud security',
   'Agile',
   'JIRA',
   'Cloud governance',
   'Cloud Custodian',
   'Open Policy Agent Rego',
   'Cloud Security/Cybersecurity',
   'Scaled Agile Framework (SAFe)',
   'AWS Certifications'],
  'description': 'We are looking for a Software/DevOps engineer with the skills to automate repeatable processes to improve the efficiency of our cloud governance capabilities, help develop integrated solutions, and assist in the delivery of policy-as-code across a multi-cloud ecosystem.'}]

In [33]:
import pandas as pd
df=pd.read_csv('companies_portfolio.csv')   
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [34]:
import chromadb
import uuid 
client = chromadb.PersistentClient('Vector_store')
collection=client.get_or_create_collection(name="portfolio") 
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
 

if not collection.count(): 
    for _, row in df.iterrows():
        collection.add(documents=row["techstack"],
                       metadatas=[{"links":row["links"]}],  
                       ids=[str(uuid.uuid4())])    

In [35]:
links=collection.query(query_texts=["Expertise in Python","Expertise in React"],n_results=2).get('metadatas')
links 

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [36]:
job=json_res 

In [37]:
prompt_email=PromptTemplate.from_template(
    """ 
    ### JOB DESCRIPTION:
      {job_description}
    ### INSTRUCTION:
    You are Raj a business development executive at XYZ.XYZ is an AI and software based company dedicated to seamless integration of business 
    processes through the use of automated tools.
    Your job is to write a cold email to clients regarding the job mentioned above describing the capability 
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase Raj's portfolio:{link_list}
    Remember you are Raj,BDE at XYZ.
    Do not provide a PREAMBLE.
    ### EMAIL (NO PREAMBLE):
    """
) 

chain_email=prompt_email | llm
res=chain_email.invoke({"job_description":str(job),"link_list":links})
print(res.content) 


Here is the cold email:

Subject: Expert Software/DevOps Engineer for Cloud Governance Automation

Dear [Client],

I came across your company and was impressed with your commitment to leveraging technology to drive business growth. As a Business Development Executive at XYZ, I'd like to introduce our team's expertise in automating repeatable processes to improve cloud governance capabilities.

Our Software/DevOps Engineer has the skills to help you develop integrated solutions and deliver policy-as-code across a multi-cloud ecosystem. With expertise in AWS, Cloud, Python, DevOps, Azure, Terraform, and Cloud Security/Cybersecurity, we can assist in automating your cloud governance capabilities, ensuring efficiency and scalability.

Our team has a proven track record of delivering successful projects, and I'd like to share some relevant examples from my portfolio:

* Python-based automation projects: https://example.com/ml-python-portfolio
* Python-based cloud integration projects: https